There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like .csv, .tsv, and .xlsx.

data/airbnb_price.csv This is a CSV file containing data on Airbnb listing prices and locations.

data/airbnb_room_type.xlsx This is an Excel file containing data on Airbnb listing descriptions and room types.

data/airbnb_last_review.tsv This is a TSV file containing data on Airbnb host names and review dates.

# **Goals**
What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.

How many of the listings are private rooms? Save this into any variable.
What is the average listing price? Round to the nearest two decimal places and save into a variable.

Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.

# **Import necessary packages**

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

# **Importing the data**

In [8]:
price_df=pd.read_csv('airbnb_price.csv')
price_df.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"""Manhattan, Midtown"""
1,3831,89 dollars,"""Brooklyn, Clinton Hill"""
2,5099,200 dollars,"""Manhattan, Murray Hill"""
3,5178,79 dollars,"""Manhattan, Hell's Kitchen"""
4,5238,150 dollars,"""Manhattan, Chinatown"""


In [9]:
room_df=pd.read_csv('airbnb_room_type.csv')
room_df.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [10]:
review_df=pd.read_csv('airbnb_last_review.csv',parse_dates=['last_review'])
review_df['last_review'] = pd.to_datetime(review_df['last_review'], format='%B %d %Y')
review_df.head()

,listing_id,host_name,last_review
0,2595,Jennifer,2019-05-21
1,3831,LisaRoxanne,2019-07-05
2,5099,Chris,2019-06-22
3,5178,Shunichi,2019-06-24
4,5238,Ben,2019-06-09


In [11]:
print(price_df.shape)
print(room_df.shape)
print(review_df.shape)

(25209, 3)
(25209, 3)
(25209, 3)


In [12]:
#merging the 3 dataframes
df=price_df.merge(room_df,on='listing_id',how='outer')
df=df.merge(review_df,on='listing_id',how='outer')
df.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"""Manhattan, Midtown""",Skylit Midtown Castle,Entire home/apt,Jennifer,2019-05-21
1,3831,89 dollars,"""Brooklyn, Clinton Hill""",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,2019-07-05
2,5099,200 dollars,"""Manhattan, Murray Hill""",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,2019-06-22
3,5178,79 dollars,"""Manhattan, Hell's Kitchen""",Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150 dollars,"""Manhattan, Chinatown""",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,2019-06-09


# **Data exploration**

In [13]:
df.memory_usage().sum()

1411832

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   price        25209 non-null  object        
 2   nbhood_full  25209 non-null  object        
 3   description  25199 non-null  object        
 4   room_type    25209 non-null  object        
 5   host_name    25201 non-null  object        
 6   last_review  25209 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.3+ MB


In [15]:
df.describe()

,listing_id,last_review
count,2.520900e+04,25209
mean,2.068922e+07,2019-05-30 07:58:41.171010560
min,2.595000e+03,2019-01-01 00:00:00
25%,1.202273e+07,2019-05-23 00:00:00
50%,2.234391e+07,2019-06-18 00:00:00
75%,3.037669e+07,2019-06-29 00:00:00
max,3.645581e+07,2019-07-09 00:00:00
std,1.102928e+07,NaN


In [16]:
df.isna().sum()

listing_id      0
price           0
nbhood_full     0
description    10
room_type       0
host_name       8
last_review     0
dtype: int64

In [17]:
df.dropna(inplace=True)

In [18]:
df.duplicated().sum()

0

# **Data manipulation**

In [19]:
df['price']=df['price'].astype('str')
df['price']=df['price'].str.replace('dollars','')
df['price']=df['price'].astype('int16')

In [20]:
df['listing_id']=df['listing_id'].astype('int32')

In [21]:
df['room_type'].value_counts()

room_type
Entire home/apt    8455
Private room       7233
entire home/apt    2664
private room       2244
ENTIRE HOME/APT    2143
PRIVATE ROOM       1865
Shared room         380
shared room         110
SHARED ROOM          97
Name: count, dtype: int64

In [22]:
df['room_type']=df['room_type'].astype('str')
df['room_type']=df['room_type'].apply(lambda x : x.lower())
df['room_type'].value_counts()

room_type
entire home/apt    13262
private room       11342
shared room          587
Name: count, dtype: int64

In [23]:
df['room_type']=df['room_type'].astype('category')
df['room_type']=df['room_type'].cat.set_categories(new_categories=['entire home/apt','private room','shared room'])
df['room_type'].cat.categories

Index(['entire home/apt', 'private room', 'shared room'], dtype='object')

In [24]:
df['nbhood_full']=df['nbhood_full'].astype('category')
df['nbhood_full'].cat.categories

Index(['"Bronx, Allerton"', '"Bronx, Baychester"', '"Bronx, Belmont"',
       '"Bronx, Bronxdale"', '"Bronx, Castle Hill"', '"Bronx, City Island"',
       '"Bronx, Claremont Village"', '"Bronx, Clason Point"',
       '"Bronx, Co-op City"', '"Bronx, Concourse Village"',
       ...
       '"Staten Island, Shore Acres"', '"Staten Island, Silver Lake"',
       '"Staten Island, South Beach"', '"Staten Island, St. George"',
       '"Staten Island, Stapleton"', '"Staten Island, Todt Hill"',
       '"Staten Island, Tompkinsville"', '"Staten Island, Tottenville"',
       '"Staten Island, West Brighton"', '"Staten Island, Willowbrook"'],
      dtype='object', length=217)

In [25]:
df['description']=df['description'].astype('str')
df['description']=df['description'].apply(lambda x: x.lower())

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25191 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25191 non-null  int32         
 1   price        25191 non-null  int16         
 2   nbhood_full  25191 non-null  category      
 3   description  25191 non-null  object        
 4   room_type    25191 non-null  category      
 5   host_name    25191 non-null  object        
 6   last_review  25191 non-null  datetime64[ns]
dtypes: category(2), datetime64[ns](1), int16(1), int32(1), object(2)
memory usage: 1018.5+ KB


In [27]:
df.memory_usage().sum()

1042995

# **Answering the questions**

In [28]:
earliest_review=df['last_review'].min()
recent_review=df['last_review'].max()
average_price=df['price'].mean().round(2)
df['room_type'].value_counts()

room_type
entire home/apt    13262
private room       11342
shared room          587
Name: count, dtype: int64

In [29]:
private_room_number=11356

In [30]:
dic = {'first_reviewed':earliest_review,'last_reviewed':recent_review,'nb_private_rooms':private_room_number,'avg_price':average_price}
review_dates=pd.DataFrame([dic])
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.82
